In [36]:
from keras._tf_keras.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [37]:
base_dir = r'C:\Users\talal\Downloads\TIRES'
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   validation_split = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [38]:
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size = (180, 101),
    batch_size = 32,
    class_mode = 'categorical',
    subset = 'training'
)

Found 2402 images belonging to 4 classes.


In [39]:
validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size = (180, 101),
    batch_size = 32,
    class_mode = 'categorical',
    subset = 'validation'
)

Found 598 images belonging to 4 classes.


In [40]:
# Build a simple CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(180, 101, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),  # Flatten the output from the convolutional layers
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')  # 4 output classes
])

In [41]:
# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)


In [42]:
#Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=25,  # Number of epochs to train
    validation_data=validation_generator
)


Epoch 1/25


c:\Users\talal\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


75/75 ━━━━━━━━━━━━━━━━━━━━ 18s 219ms/step - accuracy: 0.6851 - loss: 0.7648 - val_accuracy: 0.7986 - val_loss: 0.6241
Epoch 2/25
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8750 - loss: 0.3900 - val_accuracy: 0.8182 - val_loss: 0.7124
Epoch 3/25


c:\Users\talal\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


75/75 ━━━━━━━━━━━━━━━━━━━━ 17s 211ms/step - accuracy: 0.9321 - loss: 0.2020 - val_accuracy: 0.8698 - val_loss: 0.4066
Epoch 4/25
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 772us/step - accuracy: 1.0000 - loss: 0.0483 - val_accuracy: 0.8182 - val_loss: 0.6876
Epoch 5/25
75/75 ━━━━━━━━━━━━━━━━━━━━ 17s 207ms/step - accuracy: 0.9686 - loss: 0.1296 - val_accuracy: 0.9306 - val_loss: 0.1776
Epoch 6/25
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 839us/step - accuracy: 1.0000 - loss: 0.0168 - val_accuracy: 0.8636 - val_loss: 0.4382
Epoch 7/25
75/75 ━━━━━━━━━━━━━━━━━━━━ 17s 208ms/step - accuracy: 0.9614 - loss: 0.1234 - val_accuracy: 0.9531 - val_loss: 0.1179
Epoch 8/25
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 0.0109 - val_accuracy: 1.0000 - val_loss: 0.0339
Epoch 9/25
75/75 ━━━━━━━━━━━━━━━━━━━━ 17s 214ms/step - accuracy: 0.9802 - loss: 0.0854 - val_accuracy: 0.9479 - val_loss: 0.1370
Epoch 10/25
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 773us/step - accuracy: 1.0000 - loss: 0.0074 - val_accuracy: 1.0000 - v

In [43]:
model.save('tire_classification_model.keras')

In [44]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

In [45]:
model = load_model('tire_classification_model.keras')

In [52]:
from PIL import Image

def preprocess_image(img_path, target_size=(180, 101)):
    img = Image.open(img_path)
    img = img.resize(target_size)
    img_array = image.img_to_array(img) / 255.0  # Normalize the image
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array


In [69]:
class_labels = {0: 'SIDE', 1: 'OCR', 2: 'MultiTires', 3: 'Front'}
def classify_image(img_path):
    # Preprocess the image
    img_array = preprocess_image(img_path, target_size=(180, 101))
    
    # Make a prediction
    prediction = model.predict(img_array)
    predicted_class =  np.argmax(prediction, axis=1)[0]
    # Translate numerical label to text label
    return class_labels[predicted_class]

In [72]:
import shutil
import os
def classify_images_in_folder(input_folder, output_base_folder):
    # Ensure output subdirectories exist
    for class_name in class_labels.values():
        os.makedirs(os.path.join(output_base_folder, class_name), exist_ok=True)
    
    # Iterate through all images in the input folder
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(input_folder, filename)
            predicted_class = classify_image(img_path)
            output_folder = os.path.join(output_base_folder, predicted_class)
            shutil.move(img_path, os.path.join(output_folder, filename))
            print(f'Classified {filename} as {predicted_class} and moved to {output_folder}')

In [73]:
# Paths to directories
input_folder = r'C:\Users\talal\Downloads\random'
output_base_folder = r'C:\Users\talal\Downloads\random\classified'

# Classify all images in the input folder
classify_images_in_folder(input_folder, output_base_folder)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Classified 14038843825.JPG as SIDE and moved to C:\Users\talal\Downloads\random\classified\SIDE
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Classified 140422934160.JPG as SIDE and moved to C:\Users\talal\Downloads\random\classified\SIDE
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Classified 140423293635.JPG as Front and moved to C:\Users\talal\Downloads\random\classified\Front
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Classified 140424023629.JPG as SIDE and moved to C:\Users\talal\Downloads\random\classified\SIDE
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Classified 140424790626.JPG as OCR and moved to C:\Users\talal\Downloads\random\classified\OCR
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Classified 140441004586.JPG as SIDE and moved to C:\Users\talal\Downloads\random\classified\SIDE
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Classified 140483604855.JPG as OCR and moved to C:\Users\talal\Downloads\random\classified\OCR
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Classified 140484913